
# HW 1 - Nigel Li (nl2992)
Implement the newton's metehod to compute the implied yield. Utilising Brent's Method from the SciPy package as a benchmark,

The code below is an example of using Brent's method using the [0,20] bracket. The methods comapred for 1000 random cases were: 

**Baseline**
- `brentq`: bracketed root-finder on `[0, 20]`. Very reliable.

**SciPy Newton-family (for comparison)**
- `spop.newton(..., x0, x1, fprime=None)`: secant method.
- `spop.newton(..., x0, fprime=root_prime)`: Newton’s method with derivatives. 

These were overall, quite slow, and not optimised for purpose.


### 3 optimisation ideas ##

Overall , three general ideas are explored that is, 

1) **Manual Newton Method**
**Problem:** Newton computed the dicount separately for the f & f'. We optimsied it in such a way, that it reduces iterations, and re-uses the disc, each time.

2) **Bracketed Newton, using bisection**
**Problem:** Plain Newton can sometimes take a step that is too large (or heads toward the boundary `r -> -1`), which can cause slow convergence or divergence. This is especially risky because `(1+r)` must remain positive when `tk` is non-integer. A new update rule was created that is bounded using the `(lo + high)/2`, which is effectively akin to a `binary search`.

3) **Better starting values for SciPy secant (multiple `(x0, x1)` attempts)**
**Problem:** Secant/Newton-family methods are sensitive to initial guesses. A poor choice of `(x0, x1)` can increase iterations or cause failure. We use several potential "scale- aware" pairs and tries the first one that converges.


| Method | Idea | Failures | Wall time (ms) |
|---|---|---:|---:|
| `brentq` (benchmark) | Bracketed root-finding on `[0, 20]` | 0 | **91.6** |
| `fused_newton` (our #1) | Manual Newton; compute `f` and `f'` together (reuse discount factors) | 0 | **49.6** |
| `fused_hybrid_newton` (our #2) | Bracketed Newton; Newton step if safe, else bisection | 0 | **76.1** |
| `scipy_secant_better_starts` (our #3) | Secant with multiple better `(x0, x1)` starts | 0 | **140** |



## Conclusion
- The main performance bottleneck in Newton-style methods is repeatedly computing `(1+r)^(-tk)`.
- The best improvement was the **manual fused Newton**, which computes the discount factors once per iteration and reuses them for both `f` and `f'`.


In [35]:
import numpy as np
import scipy.optimize as spop
import timeit

## What problem is this solving?

You have cashflows `c_k` paid at times `t_k`.

For a yield `r`, the present value (price from the model) is:

`P(r) = sum_{k=1..n} c_k / (1 + r)^(t_k)`

The market price is `p`.

**Yield to maturity (YTM)** is the value of `r` that makes the model price match the market price:

`P(r) = p`

## Why this is a root-finding problem

Define:

`f(r) = P(r) - p`

Then YTM is the `r` such that:

`f(r) = 0`

So the code is solving: **find the root of f(r)** (using a numerical method like Newton's & Halley's)



In [4]:
# NPV function. tk and ck are the array of time and cashflow.
# Sanity Check: length of tk and ck should be the same, as we want to ensure that each cashflow is matched to time.
# NPV is return as the summation of all discounted cashflows of sum (ck[k] / (1 +r) ** tk[k]) for k in range(len(tk))
def npv(r, tk, ck):
  assert(len(tk) == len(ck))
  return np.sum(np.power(1.0 + r, -tk) * ck)

In [5]:
# Condition for root, such that it will return an r such that npv(r, tk, ck) - p = 0, which is equivalent to npv(r, tk, ck) = p
def root(r, p, tk, ck):
  return npv(r, tk, ck) - p

In [6]:
# Seed for reproducibility
rng = np.random.default_rng(12345)

# Sample Approach w brentQ:

In [ ]:
%%time
for k in range(1000):
  # Randomly generating cashflow, time, and price
  ck = rng.uniform(0, 10, 100) # Cashflow between 0 and 10, 100 # in total.
  ck[-1] += 100 # Final Cash flow + 100, so its like a final repayment
  tk = np.cumsum(rng.uniform(0.1, 0.3, 100)) # Setting up time between 0.1, to 0.3, so that we have increasing time
  p = rng.uniform(10.0, 100) # some px between 10 and 100;

  #### Instead of `brentq` below, implement Newton's method.
  args = (p, tk, ck)

  # scipy.optimize.brentq(f, a, b, args=(), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
  r = spop.brentq(root, 0.0, 20.0, args=args)
  assert(np.abs(root(r, p, tk, ck))<1e-8)

CPU times: user 49.9 ms, sys: 1.3 ms, total: 51.2 ms
Wall time: 51.3 ms


# Sample Newton Method (using scipy) to compare with BrentQ
- Probably should've made above set-up as a function but oh well.....
- FROM NOW ON, WE INTRODUCE A NEW SET OF RANDOMISED DATA with seed(12345)

In [22]:
# Defining the root function's derivative:
# f'(r) = sum (ck * (-tk) * (1 + r) ** (-tk - 1)) for k in range(len(tk))
def root_prime(r, p, tk, ck):
  return np.sum(-tk * np.power(1.0 + r, -tk - 1) * ck)

In [23]:
# intiial guess based on the cash-flow weighted time (like macaulay duration), and then approcimated the entire stream ass a single payment PV0
def initial_guess(p, tk, ck):
    PV0 = np.sum(ck)
    Tbar = np.sum(tk * ck) / PV0
    r0 = (PV0 / p) ** (1.0 / Tbar) - 1.0
    # keep 1+r positive (domain of (1+r)^(-t))
    return max(r0, -0.95)

In [24]:
rng = np.random.default_rng(12345)

cases = []
for k in range(1000):
    ck = rng.uniform(0, 10, 100)
    ck[-1] += 100
    tk = np.cumsum(rng.uniform(0.1, 0.3, 100))
    p = rng.uniform(10.0, 100)
    cases.append((p, tk, ck))

# General brentQ

In [25]:
%%time
for (p, tk, ck) in cases:
    r = spop.brentq(root, 0.0, 20.0, args=(p, tk, ck))
    assert abs(root(r, p, tk, ck)) < 1e-8


CPU times: user 47.2 ms, sys: 2.03 ms, total: 49.3 ms
Wall time: 49.6 ms


# Secant mode:

In [ ]:
%%time
for (p, tk, ck) in cases:
    r0 = initial_guess(p, tk, ck)
    r1 = r0 + 0.1 * (1.0 + abs(r0)) 
    r = spop.newton(root, x0=r0, x1=r1, fprime=None, args=(p, tk, ck), tol=1e-8, maxiter=80)
    assert abs(root(r, p, tk, ck)) < 1e-8



CPU times: user 98.7 ms, sys: 4.14 ms, total: 103 ms
Wall time: 107 ms


In [27]:
%%time
for (p, tk, ck) in cases:
    r0 = initial_guess(p, tk, ck)
    r = spop.newton(root, x0=r0, fprime=root_prime, args=(p, tk, ck), tol=1e-8, maxiter=50)
    assert abs(root(r, p, tk, ck)) < 1e-8

CPU times: user 95.3 ms, sys: 1.21 ms, total: 96.5 ms
Wall time: 96.1 ms


# Optimisations of existing Newton's Method:
1) Do Newton manually and compute f and f' together (reuse the same powers once per step).
- Shows as `fused_newton()`, such that each iteration computed the disc once, and f & f'  in re-using disc, and then using the update rule. The SciPy Newton with derivatives calls the root() fx and computes all powers, then the root_prime() again, so it does `np.power()` twice per iteration.

2) Keep a bracket and only take a Newton step if it stays inside; otherwise do a bisection step (effecitvley a binary search)
- In this case, we started with a bracket, and computed f & f', and used the same update rule. If r_new leaves the bracket initially proposed, it is replaced with the bisection (like a binary search), which does `r_new = (lo + hi)/2` such that we have f_new. 

3) Use better starting values: clamp r0 away from -1 and pick x1 based on scale (not a fixed bump).
- Utilised Secant model again but tried multiple x0, 1 pairs, and returns the first one that converges ()

In [28]:
def better_initial_guess(p, tk, ck):
    PV0 = np.sum(ck)
    Tbar = np.sum(tk * ck) / PV0
    r0 = (PV0 / p) ** (1.0 / Tbar) - 1.0
    return float(np.clip(r0, -0.90, 20.0))

In [29]:
def fused_newton(p, tk, ck, r0, tol=1e-8, maxiter=50):
    r = float(r0)
    for _ in range(maxiter):
        if r <= -0.999999999:
            raise RuntimeError("Out of domain (1+r<=0).")

        onepr = 1.0 + r
        disc = np.power(onepr, -tk)              # (1+r)^(-tk)
        f = np.sum(ck * disc) - p
        if abs(f) < tol:
            return r

        # f'(r) = sum ck * (-tk) * (1+r)^(-tk-1) = sum ck * (-tk) * disc / (1+r)
        fp = np.sum(ck * (-tk) * disc) / onepr
        if fp == 0.0 or not np.isfinite(fp):
            raise RuntimeError("Bad derivative.")

        r -= f / fp

    raise RuntimeError("No convergence.")

In [30]:
def fused_hybrid_newton(p, tk, ck, lo=0.0, hi=20.0, r0=None, tol=1e-8, maxiter=80):
    f_lo = root(lo, p, tk, ck)
    f_hi = root(hi, p, tk, ck)
    if f_lo == 0.0:
        return lo
    if f_hi == 0.0:
        return hi
    if f_lo * f_hi > 0:
        raise RuntimeError("No sign change in bracket.")

    r = 0.5 * (lo + hi) if r0 is None else float(np.clip(r0, lo, hi))

    for _ in range(maxiter):
        # compute fused f and f'
        onepr = 1.0 + r
        if onepr <= 0:
            r = 0.5 * (lo + hi)
            onepr = 1.0 + r

        disc = np.power(onepr, -tk)
        f = np.sum(ck * disc) - p
        if abs(f) < tol:
            return r

        fp = np.sum(ck * (-tk) * disc) / onepr
        # Newton candidate
        use_newton = np.isfinite(fp) and fp != 0.0
        if use_newton:
            r_new = r - f / fp
        else:
            r_new = np.nan

        # If Newton step goes out of bracket, use bisection
        if (not np.isfinite(r_new)) or (r_new <= lo) or (r_new >= hi):
            r_new = 0.5 * (lo + hi)

        f_new = root(r_new, p, tk, ck)

        # update bracket
        if f_lo * f_new <= 0:
            hi, f_hi = r_new, f_new
        else:
            lo, f_lo = r_new, f_new

        r = r_new

    raise RuntimeError("No convergence.")

In [ ]:
def scipy_secant_better_starts(p, tk, ck, tol=1e-8, maxiter=80):
    r0 = initial_guess(p, tk, ck)
    pairs = [
        (r0, r0 + 0.1 * (1.0 + abs(r0))),
        (max(r0, 0.0), max(r0, 0.0) + 0.25),
        (0.02, 0.25),
        (0.10, 0.60),
    ]
    last = None
    for x0, x1 in pairs:
        try:
            r = spop.newton(root, x0=x0, x1=x1, args=(p, tk, ck), tol=tol, maxiter=maxiter)
            if abs(root(r, p, tk, ck)) < 1e-8:
                return r
        except RuntimeError as e:
            last = e
    raise RuntimeError(f"Secant failed. Last: {last}")

In [37]:
%%time
for (p, tk, ck) in cases:
    r = spop.brentq(root, 0.0, 20.0, args=(p, tk, ck))
    assert abs(root(r, p, tk, ck)) < 1e-8

CPU times: user 55.4 ms, sys: 17.5 ms, total: 72.9 ms
Wall time: 91.6 ms


In [38]:
%%time
fails_1 = 0
for (p, tk, ck) in cases:
    try:
        r0 = initial_guess(p, tk, ck)
        r = fused_newton(p, tk, ck, r0, tol=1e-8, maxiter=50)
        assert abs(root(r, p, tk, ck)) < 1e-8
    except Exception:
        fails_1 += 1
print("fails fused_newton:", fails_1)


fails fused_newton: 0
CPU times: user 45.2 ms, sys: 3.5 ms, total: 48.7 ms
Wall time: 49.6 ms


In [39]:
%%time
fails_2 = 0
for (p, tk, ck) in cases:
    try:
        r0 = initial_guess(p, tk, ck)
        r = fused_hybrid_newton(p, tk, ck, lo=0.0, hi=20.0, r0=r0, tol=1e-8, maxiter=80)
        assert abs(root(r, p, tk, ck)) < 1e-8
    except Exception:
        fails_2 += 1
print("fails fused_hybrid_newton:", fails_2)


fails fused_hybrid_newton: 0
CPU times: user 73.4 ms, sys: 2.71 ms, total: 76.1 ms
Wall time: 76.1 ms


In [40]:
%%time
fails_3 = 0
for (p, tk, ck) in cases:
    try:
        r = scipy_secant_better_starts(p, tk, ck, tol=1e-8, maxiter=80)
        assert abs(root(r, p, tk, ck)) < 1e-8
    except Exception:
        fails_3 += 1
print("fails scipy_secant_better_starts:", fails_3)

fails scipy_secant_better_starts: 0
CPU times: user 119 ms, sys: 5.89 ms, total: 125 ms
Wall time: 140 ms
